## Fazendo import das dependências

In [1]:
from BHA_functions import asyncSimulations_Linux
from BHA_functions import DataCollector, DataGroup
from BHA_functions import GraphicGenerator
from datetime import datetime

## Definindo as constantes das simulações

In [2]:
PATH: str = 'Simulations_Data/topology_simulations'
CORES: int = 12
RUNS_PER_POINT: int = 1

TARGET: tuple[bool] = (True, False)

NETWORK_PROB: float = 0.8
BHA_PROB: float = 0.4

TOPOLOGIES: str = ('Grade', 'Ba', 'Er')
TOPOLOGY_PARAMS: tuple[float] = (0.1, 0.3, 0.5)

PROB_EDGE_CREATION: float = 0.4
EDGES_TO_ATTACH: int = 3

PASS_OF_NODE: int = 12
NUMBER_OF_NODES: tuple = tuple([i for i in range(12, 108, PASS_OF_NODE)])
GRADE_NODES: dict = {
    12:(3, 4),
    24:(4, 6),
    36:(6, 6),
    48:(6, 8),
    60:(6, 10),
    72:(8, 9),
    84:(7, 12),
    96:(8, 12)
}

GRAPHIC_POINTS: int = len(NUMBER_OF_NODES)

## Rodando a simulação sem Black Hole em todas as topologias

In [ ]:
# Default params
simulations_params: dict = {
            'runs':RUNS_PER_POINT,
            'topology':TOPOLOGIES[0],
            'number_nodes':20,
            'rows':4,
            'columns':3,
            'prob_edge_creation':PROB_EDGE_CREATION,
            'edges_to_attach':EDGES_TO_ATTACH,
            'entanglements_replanished':10,
            'requests':100,
            'attempts_per_request':2,
            'network_prob':NETWORK_PROB,
            'num_black_holes':0,
            'black_hole_prob':BHA_PROB,
            'black_hole_target':TARGET
        }

for topology in TOPOLOGIES:
    simulations_params['topology'] = topology

    for number_of_nodes in NUMBER_OF_NODES:
        simulations_params['number_nodes'] = number_of_nodes

        if topology == 'Grade':
            simulations_params['rows'] = GRADE_NODES[number_of_nodes][0]
            simulations_params['columns'] = GRADE_NODES[number_of_nodes][1]

        # Runing no BHA simulation
        default_simulation: DataCollector = asyncSimulations_Linux(cores=CORES, **simulations_params)
        default_simulation.save(file_name=f'{PATH}/Default_Network/{topology}')

## Realizando todas as simulações com Black Holes

In [ ]:
# Run all topologies and update your parameter
for topology in TOPOLOGIES:
    simulations_params['topology'] = topology
    
    # Run all targets (True, False) and update your parameter
    for target in TARGET:
        simulations_params['target'] = target
        
        # Run all topology parameters (0.1, 0.3, 0.5) and updating your parameters
        for indexgroup, topology_param in enumerate(TOPOLOGY_PARAMS):
            
            # Grade topology won't change your params, so no need run more than once simulation per target
            if topology == "Grade" and indexgroup > 0:
                continue

            # Updating parameters of topology (0.1, 0. 3 or 0.5) in prob_edge_creation and (1, 3 or 5) in edges_to_attach
            simulations_params['prob_edge_creation'] = topology_param
            simulations_params['edges_to_attach'] = int(topology_param * 10)

            # Run all points and update your parameter
            for point, number_of_nodes in enumerate(NUMBER_OF_NODES):
                start: datetime = datetime.now() # Debug: start timer to BenchMark
                
                # Update the topology parameters, because number_nodes don't affect this topology
                if topology == 'Grade':
                    simulations_params['rows'] = GRADE_NODES[number_of_nodes][0]
                    simulations_params['columns'] = GRADE_NODES[number_of_nodes][1]

                # Updating umber of nodes of simulation
                simulations_params['num_black_holes'] = int(number_of_nodes * topology_param)

                # Running simulation
                simulations_dc: DataCollector = asyncSimulations_Linux(cores=CORES, **simulations_params)

                # Saving CSV file
                simulations_dc.save(file_name=f'{PATH}/BHA_Network/{topology}/Target-{target}/{int(topology_param*10)}param/point{point}')

                # BenchMark
                print(f"As {simulations_params['runs']} simulações da topologia {topology} finalizaram no tempo de: {datetime.now()-start}")
                print('-='*50)

## Coletando os dados diretamente dos CSVs de cada simulação

In [ ]:
default_networks: dict = dict()
for topology in TOPOLOGIES:
    default_network: DataCollector = DataCollector()
    default_network.get_DataFrame_csv(f'{PATH}/Default_Network/{topology}.csv')
    default_networks[topology] = default_network

# Create DataGroups for all params and Topologys
dg_grade_target_off: DataGroup = DataGroup()
dg_grade_target_on: DataGroup = DataGroup()
dg_ba_target_off: DataGroup = DataGroup()
dg_ba_target_on: DataGroup = DataGroup()
dg_er_target_off: DataGroup = DataGroup()
dg_er_target_on: DataGroup = DataGroup()

# Puting all DataGroups in a dict
dg_dict: dict = {
    'Grade':{True:dg_grade_target_on, False:dg_grade_target_off},
    'Ba':{True:dg_ba_target_on, False:dg_ba_target_off},
    'Er':{True:dg_er_target_on, False:dg_er_target_off}
}

# Collecting data of all topologies, except Grade topology
# The DataGroup and dict will look like this
# dg_dict = {
# 'Grade': {True:[(D1,..., Dnpoints)], False: [(D1,..., Dnpoints)]}, 
# 'Ba': {True:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)], False:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)]},
# 'Er': {True:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)], False:[(D1,..., Dnpoints), (D1,..., Dnpoints), (D1,..., Dnpoints)]}
# }
# D1 and Dnpoints are DataGroups with all data of current point
#
# {Topology: { True: [ (     D1, ..., Dn), ..., ()], False:[ (),..., ()]}, ...}
# ↑          ↑         ↑     ↑
# ↑          ↑         ↑     DataCollector per point
# ↑          ↑        Param of simulation (0.1, 0.3, 0.5)
# ↑         Dict of tagerts on (True) or off (False)
# Dict of topology

for topology in TOPOLOGIES:
    if topology == "Grade":
        continue
    for target in TARGET:
        for indexgroup, topology_param in enumerate(TOPOLOGY_PARAMS):
            for point in range(0, GRAPHIC_POINTS):
                temp_dc: DataCollector = DataCollector()
                temp_dc.get_DataFrame_csv(f'{PATH}/BHA_Network/{topology}/Target-{target}/{int(topology_param*10)}param/point{point}.csv')
                
                dg_dict[topology][target].add_Data(value=temp_dc, indexgroup=indexgroup)

# Collecting only data of Grade topology        
for target in TARGET:
    dg_dict['Grade'][target].add_Group(value=tuple())
    for point in range(0, GRAPHIC_POINTS):
        temp_dc: DataCollector = DataCollector()
        temp_dc.get_DataFrame_csv(f'{PATH}/BHA_Network/Grade/Target-{target}/point{point}.csv')

        dg_dict["Grade"][topology].add_Data(value=temp_dc, indexgroup=0)


## Criando uma função genérica para plotar os gráficos

In [ ]:
def plotGraphic(
        dataGroup: DataGroup,
        default_diff: dict[DataCollector], 
        topology: str,
        pass_of_node: float,
        topology_params: tuple,
        y_column_name: str, 
        title: str, 
        x_label: str, 
        y_label: str) -> None:
    """
    Will show the simulations graph

    Args:
        dataGroup (required): DataGroup with all DataCollectors
        default_diff (required): Pass the network without black holes to compare, if None don't affect the result
        topology (required): String with name of the selected topology
        pass_of_node (required): Variation of nodes per point
        topology_params (required): Tuple with all parameter of each plot
        y_column_name (required): Name of column data
        title (required): Title of graph
        x_label (required): X axis label
        y_label (required): Y axis label
    """
    
    graphicGen: GraphicGenerator = GraphicGenerator()

    for pos, datacollectors in enumerate(dataGroup):

        if topology == 'Ba':
            plot_label: str = f"m={int(topology_params[pos])}"
        elif topology == 'Er':
            plot_label: str = f"p={topology_params[pos]}"
        else:
            plot_label: str = ''

        graphicGen.add_on_plot(
            plot_label=plot_label, 
            x_column=(12, pass_of_node), 
            y_column_name=y_column_name, 
            y_standard_deviation=False,
            dc=datacollectors,
            default_diff=default_diff[topology])

    graphicGen.show_plot(
        title=title,
        x_label=x_label, 
        y_label=y_label)

## Gerando o gráfico da Topologia de Grade sem alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Grade'][False],
    default_diff=default_network,
    topology='Grade',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Grade)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
    )

## Gerando o gráfico da Topologia de Grade com alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Grade'][True],
    default_diff=default_network,
    topology='Grade',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Grade)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
    )

## Gerando o Gráfico da Topologia Barabasi-Albert sem alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Ba'][False],
    default_diff=default_network,
    topology='Ba',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Barabasi-Albert)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
)

## Gerando o Gráfico da Topologia Barabasi-Albert com alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Ba'][True],
    default_diff=default_network,
    topology='Ba',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Barabasi-Albert)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
)

## Gerando o Gráfico da Topologia Erdos Renyi sem alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Er'][False],
    default_diff=default_network,
    topology='Er',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Erdos Renyi)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
)

## Gerando o Gráfico da Topologia Erdos Renyi com alvo

In [ ]:
plotGraphic(
    dataGroup=dg_dict['Er'][True],
    default_diff=default_network,
    topology='Er',
    pass_of_node=PASS_OF_NODE,
    topology_params=TOPOLOGY_PARAMS,
    y_column_name="Success Tax",
    title="Diferença na Taxa de Sucesso vs Número de nós (Erdos Renyi)",
    x_label='Número de nós',
    y_label='Diferença na Taxa de Sucesso'
)